# Setup
1. install python (3.8) and git
2. install virtualenv e.g. with `<path to python> -m to pip install virtualenv`
3. create a virtual environment that uses python 3.8 using virtualenv e.g. with `<path to python> -m virtualenv "<path to virtual environment>"`
4. activate the virtual environment e.g. on windows with `.\<pathto env>\Scripts\activate`
5. install vscode and python extension
6. git clone brainlit (https://github.com/neurodata/brainlit) and neuroglancer (https://github.com/google/neuroglancer)
7. install brainlit from source e.g. `pip install -e .` in brainlit folder 

In [1]:
from brainlit.utils.write import czi_to_zarr, zarr_to_omezarr
import zarr
from cloudvolume import CloudVolume
import json
from pathlib import Path

# Convert czi to zarr

In [2]:
project_path = "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/"  # "C:\\Users\\Sriram Sudarsanam\\Desktop\\NeuroglancerTrial\\"
czi_path = f"{project_path}test.czi"  # path to czi image
out_dir = f"{project_path}"  # path to directory where zarr should be made, should end in slash

In [8]:
zarr_paths = czi_to_zarr(czi_path=czi_path, out_dir=out_dir, fg_channel=1, parallel=4)

Writing 2 zarrs of shape 1998x2009x40 from czi with dims [{'X': (0, 2009), 'Y': (0, 1998), 'Z': (0, 40), 'C': (0, 2), 'T': (0, 1), 'V': (0, 1), 'B': (0, 1)}]


Saving slices background...: 100%|██████████| 4/4 [00:00<00:00, 2512.69it/s]


# Convert zarr to ome-zarr

In [9]:
out_path = f"{project_path}fg_ome.zarr"  # path of ome zarr to be made

In [10]:
zarr_to_omezarr(zarr_path=zarr_paths[0], out_path=out_path)

Converting /Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/fg.zarr to ome-zarr


# View ome-zarr on neuroglancer

1. Use https://github.com/google/neuroglancer/blob/master/cors_webserver.py to serve the data e.g. `python cors_webserver.py -d "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data" -p 9010`
2. Launch neuroglancer app e.g. https://neuroglancer-demo.appspot.com
3. Add layer to neuroglancer app e.g. zarr://http://127.0.0.1:9010/sriram-adipo-brain1-im3/fg_ome.zarr

# Add trace layer

In [ ]:
traces_dir = f"{out_dir}traces/"
z = zarr.open(out_dir + "fg.zarr")
volume_size = z.shape
chunk_size = z.chunks
resolution = [1000000000, 1000000000, 1000000000]

outpath = f"precomputed://file://{traces_dir}"

info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="segmentation",
    data_type="uint16",
    encoding="raw",
    resolution=resolution,  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=chunk_size,  # units are voxels
    volume_size=volume_size,  # e.g. a cubic millimeter dataset
    skeletons="skeletons",
)
vol = CloudVolume(outpath, info=info, compress=False)
vol.commit_info()
vol.skeleton.meta.commit_info()

# remove vertex type attribute because it is a uint8 and incompatible with neuroglancer
info_path = f"{traces_dir}skeletons/info"
with open(info_path) as f:
    data = json.load(f)
    for i, attr in enumerate(data["vertex_attributes"]):
        if attr["id"] == "vertex_types":
            data["vertex_attributes"].pop(i)
            break

with open(info_path, "w") as f:
    json.dump(data, f)

# Ready to trace
Use `trace.py` to trace neurons

# Optional: Activate ViterBrain

In [3]:
from skimage import io 
from brainlit.algorithms.generate_fragments.state_generation import state_generation
import numpy as np

In [9]:
image_path = out_dir + "fg.zarr"
ilastik_program_path = "/Applications/ilastik-1.4.0b21-OSX.app/Contents/ilastik-release/run_ilastik.sh"
ilastik_project_path = out_dir + "ex_fg.ilp"
chunk_size = [z_im.chunks[0], z_im.chunks[1], 40]
data_bin = out_dir + "data_bin/"

prob_path = out_dir + "probs.zarr"
fragment_path = out_dir + "labels.zarr"
tiered_path = out_dir + "tiered.zarr"
states_path = out_dir + "nx.pickle"

sg = state_generation(image_path = image_path,
                      new_layers_dir = out_dir,
                      ilastik_program_path = ilastik_program_path,
                      ilastik_project_path = ilastik_project_path,
                      chunk_size=chunk_size,
                      parallel=6,
                    #   prob_path = prob_path,
                    #   fragment_path=fragment_path,
                    #   tiered_path=tiered_path,
                    #   states_path = states_path
                      )

In [9]:
sg.predict(data_bin=data_bin)
sg.compute_frags()
sg.compute_soma_lbls()
sg.compute_image_tiered()
sg.compute_states()
sg.compute_edge_weights()

100%|██████████| 10/10 [00:00<00:00, 46.67it/s]


Processing image of shape (1998, 2009, 40) with chunks (200, 200, 10) into probability image /Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/probs.zarr of shape (1998, 2009, 40)


Computing Ilastik Predictions: 100%|██████████| 2/2 [04:43<00:00, 141.88s/it]


# Other

In [ ]:
z_im = zarr.open(image_path)
subvol = z[:40,:40,:40]
io.imsave(out_dir + "/train.tif", subvol)

In [6]:
import pickle
from brainlit.algorithms.connect_fragments.viterbrain import explain_viterbrain

with open("/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/viterbrain.pickle", "rb") as handle:
    vb = pickle.load(handle)

In [7]:
explain_viterbrain(vb, [558, 549, 4], [480, 517, 2], [])

3 states
7 coordinates
0: [558, 549, 4] f1176 
1: [534, 510, 15] f1176 s2351
2: [526, 524, 9] f1154 s2351
Transition: 2351->2307: [(2351, 2294, None), (2351, 2295, None), (2351, 2306, None), (2351, 2307, None), (2351, 2344, None), (2351, 2346, None), (2351, 2357, None), (2351, 2373, None), (2351, 2374, None), (2351, 2380, None), (2351, 2381, None), (2351, 2382, None), (2351, 2383, None), (2351, 2386, None), (2351, 2387, None), (2351, 2388, None), (2351, 2397, None), (2351, 2399, None)]
2: [526, 524, 9] f1154 s2307
3: [512, 526, 10] f1154 s2307
Transition: 2307->2343: [(2307, 2285, None), (2307, 2294, None), (2307, 2295, None), (2307, 2296, None), (2307, 2297, None), (2307, 2310, None), (2307, 2343, None), (2307, 2344, None), (2307, 2345, None), (2307, 2346, None), (2307, 2350, None), (2307, 2373, None), (2307, 2374, None), (2307, 2375, None), (2307, 2380, None), (2307, 2381, None), (2307, 2396, None), (2307, 2397, None)]
3: [512, 526, 10] f1154 s2343
4: [498, 510, 4] f1172 s2343
5: [48

vs fragments [1176, 1173, 1172]